<a href="https://colab.research.google.com/github/sanika-mhadgut/NLP/blob/master/Topic_Modelling_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Name : Sanika Mhadgut
#Branch : Btech Data Science Sem 6
#Roll No: J031

Topic modeling is an unsupervised machine learning technique that's capable of scanning a set of documents, detecting word and phrase patterns within them, and automatically clustering word groups and similar expressions that best characterize a set of documents.

Topic Modeling is a technique to extract the hidden topics from large volumes of text. Latent Dirichlet Allocation(LDA) is a popular algorithm for topic modeling. The challenge, however, is how to extract good quality of topics that are clear, segregated and meaningful. This depends heavily on the quality of text preprocessing and the strategy of finding the optimal number of topics. 

In [0]:
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer 
import nltk
import pandas as pd
import urllib, json
nltk.download('punkt')
from google.colab import files
!pip install -q kaggle

uploaded = files.upload()

#The core packages used in this tutorial are re, gensim, spacy and pyLDAvis. Besides this we will also using matplotlib, numpy and pandas for data handling and visualization. 


Saving kaggle.json to kaggle (1).json


In [0]:
!cp kaggle.json ~/.kaggle/kaggle.json

In [0]:
!kaggle datasets download -d wcukierski/enron-email-dataset

 96% 345M/358M [00:06<00:00, 63.3MB/s]
100% 358M/358M [00:06<00:00, 59.3MB/s]


In [0]:
df = pd.read_csv('enron-email-dataset.zip', compression='zip', header=0, sep=',', quotechar='"')
df=df.head(10)

In [0]:
df

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...
5,allen-p/_sent_mail/1002.,Message-ID: <30965995.1075863688265.JavaMail.e...
6,allen-p/_sent_mail/1003.,Message-ID: <16254169.1075863688286.JavaMail.e...
7,allen-p/_sent_mail/1004.,Message-ID: <17189699.1075863688308.JavaMail.e...
8,allen-p/_sent_mail/101.,Message-ID: <20641191.1075855687472.JavaMail.e...
9,allen-p/_sent_mail/102.,Message-ID: <30795301.1075855687494.JavaMail.e...


In [0]:
nltk.download("stopwords")
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize    
stop_words = set(stopwords.words('english')) 

from nltk.tokenize import RegexpTokenizer

def clean_list(sentence): 
  tokenizer = RegexpTokenizer(r'\w+') 
  word_tokens=tokenizer.tokenize(sentence)
  filtered_sentence = [] 
  for w in word_tokens: 
      if w not in stop_words:
        if w.isalpha():
          filtered_sentence.append(w) 
  return filtered_sentence

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
df["Text2"] = df['message'].apply(clean_list)
print(df["Text2"])

0    [Message, ID, JavaMail, evans, thyme, Date, Mo...
1    [Message, ID, JavaMail, evans, thyme, Date, Fr...
2    [Message, ID, JavaMail, evans, thyme, Date, We...
3    [Message, ID, JavaMail, evans, thyme, Date, Mo...
4    [Message, ID, JavaMail, evans, thyme, Date, Th...
5    [Message, ID, JavaMail, evans, thyme, Date, Th...
6    [Message, ID, JavaMail, evans, thyme, Date, Tu...
7    [Message, ID, JavaMail, evans, thyme, Date, Fr...
8    [Message, ID, JavaMail, evans, thyme, Date, Tu...
9    [Message, ID, JavaMail, evans, thyme, Date, Mo...
Name: Text2, dtype: object


In [0]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [0]:
vectors = vectorizer.fit_transform(df["message"])

tokenize each sentence into a list of words, removing punctuations and unnecessary characters altogether.

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
def dummy_fun(doc):
    return doc
vectorizer = TfidfVectorizer(analyzer='word',tokenizer=dummy_fun,preprocessor=dummy_fun,token_pattern=None)
vectors = vectorizer.fit_transform(df['Text2'])
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df3 = pd.DataFrame(denselist, columns=feature_names)

In [0]:
from sklearn.decomposition import NMF
model = NMF(n_components=5, init='nndsvd', random_state=42, alpha = 0.1, l1_ratio = 0.5)
W = model.fit_transform(df3)
H = model.components_

LDA’s approach to topic modeling is it considers each document as a collection of topics in a certain proportion. And each topic as a collection of keywords, again, in a certain proportion.

Once you provide the algorithm with the number of topics, all it does it to rearrange the topics distribution within the documents and keywords distribution within the topics to obtain a good composition of topic-keywords distribution.

In [0]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components = 10, random_state=0)
lda.fit(df3)

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()
print_top_words(lda, list(df3), 10)

Topic #0: Oct May Mon As across time phone required john Fri
Topic #1: test business meetings would trip go leah way Leah successful
Topic #2: Oct May Mon As across time phone required john Fri
Topic #3: Oct May Mon As across time phone required john Fri
Topic #4: Belden Tim IP tim EnronXGate belden forecast Here Company scott
Topic #5: morning Joyce Teixeira teixeira calls Jul review PRC joyce phone
Topic #6: price gas l johnson david Gas fixed customer deal John
Topic #7: Oct May Mon As across time phone required john Fri
Topic #8: X com enron Allen To From Content Phillip K pallen
Topic #9: Oct May Mon As across time phone required john Fri

